In [52]:
__author__ = 'racah'
from swiglpk import *
import numpy as np
import pandas as pd
import copy
import csv
# todo: make way to reconstruct data if fails (save intermediate results)
#write code to simulate best bench
#write code to add certain draft picks to your bench

In [53]:
class ff_opt(object):
    def __init__(self, row_dict={'QB': 1, 'RB': 3, 'WR': 3, 'TE': 1, 'K': 1, 'Budget': 165, 'Tot': 9},
                 keeper_file="./keepers.csv",
                 filename='data.csv',
                 df=None):
        if not df:
            self.df = pd.read_csv('data.csv')
        else:
            self.df = df
        self.row_dict = row_dict
        self.original_df = self.df
        self.my_points = 0
        aths = self.process_keepers(keeper_file)
        self.backup_options(aths)
        
    def backup_options(self,aths):
        for ath,price, pos in aths:
            new_aths = self.what_if_they_got(ath,quiet=True)
            for new_ath,price, new_pos in new_aths:
                if new_pos == pos:
                    print "backup ", pos, " ", new_ath, " $",int(price)
            
        
    def process_keepers(self,keeper_file):
        with open(keeper_file, 'rU') as csvfile:
            reader = csv.reader(csvfile, delimiter=',')
            for idx, row in enumerate(reader):
                if idx == 0:
                    fieldnames = row
                else:
                    team,player, price = row
                    player=player.lower()
                    price = int(price)
                    if team == 'Evan':
                        aths = self.i_got(player, price)
                    else:
                        aths = self.they_got(player)
        return aths


    def get_player(self,df, player_name):
        df_mask = df.player == player_name
        if not any(df_mask):
            print "That player ain't even an option"
            return None
        else:
            player = df[df_mask]
        return player


    def i_got(self, player_name, what_i_paid):
        print "i got ", player_name
        player = self.get_player(self.df,player_name)
        if player is None:
            return


        #subtract from your budget
        budget = self.row_dict['Budget']
        if budget >= what_i_paid:
            self.row_dict['Budget'] -= what_i_paid
        else:
            print "you would be out of budget dood!"
            print "current budget: ", budget
            return

        tot = self.row_dict['Tot']
        if tot > 0:
            self.row_dict['Tot'] -= 1

        #subtract from number of positions
        pos = player.pos.values[0]
        num_pos = self.row_dict[pos]
        if num_pos > 0:
            self.row_dict[pos] -= 1

        self.my_points += player.points.values[0]

        #remove player
        self.df = self.df[self.df.player != player_name]

        aths = self.run_opt()
        self.backup_options(aths)
        return aths

    def they_got(self, player_name, quiet=False):
        if not quiet:
            print "they got: ", player_name
        self.df = self.df[self.df.player != player_name]

        aths = self.run_opt(quiet=quiet)
        self.backup_options(aths)
        return aths

    def what_if_they_got(self, player_name,quiet=False):
        if not quiet:
            print "what if they got: ", player_name
        temp_df = self.df[self.df.player != player_name]

        return self.run_opt(df=temp_df,quiet=quiet)

    def what_if_i_got(self, player_name, what_i_paid, quiet=False):
        if not quiet:
            print "what if I got: ", player_name
        temp_row_dict = copy.deepcopy(self.row_dict)
        temp_df = copy.deepcopy(self.df)
        temp_my_points = copy.deepcopy(self.my_points)

        df_mask = self.df.player == player_name
        if not any(df_mask):
            print "That player ain't even an option"
            return
        else:
            player = self.df[df_mask]


        #subtract from your budget
        budget = temp_row_dict['Budget']
        if budget >= what_i_paid:
            temp_row_dict['Budget'] -= what_i_paid
        else:
            print "you would be out of budget dood!"
            print "current budget: ", budget
            return

        tot = temp_row_dict['Tot']
        if tot > 0:
            temp_row_dict['Tot'] -= 1

        #subtract from number of positions
        pos = player.pos.values[0]
        num_pos = temp_row_dict[pos]
        if num_pos > 0:
            temp_row_dict[pos] -= 1

        temp_my_points += player.points.values[0]

        #remove player
        temp_df = temp_df[temp_df.player != player_name]

        self.run_opt(df=temp_df, row_dict=temp_row_dict, my_points=temp_my_points)


    def max_i_could_pay_for(self,player_name):
        temp_df = copy.deepcopy(self.df)
        player = self.get_player(temp_df, player_name)
        lst = self.run_opt(df=temp_df)
        price = int(player.price)
        while self.player_in_list(player_name, lst):
            price += 1
            temp_df.set_value(int(np.asarray(temp_df[temp_df.player==player_name].index).astype('int64')),'price',price)
            lst = self.run_opt(df=temp_df)
        
        price = int(temp_df[temp_df.player==player_name].price) - 1
        #print lst
        print price
        return price
    
    def player_in_list(self,plyr, lst):
        for (player, price, pos) in lst:
            if plyr ==player:
                return True
        return False
                
            
    def run_opt(self, df=None, row_dict=None, my_points=None, quiet=False):
        pos_list = ['QB', 'RB', 'TE', 'WR', 'K']
        if not isinstance(df, pd.core.frame.DataFrame):
            df = self.df
        if not row_dict:
            row_dict = self.row_dict
        if not my_points:
            my_points = self.my_points

        prices = np.asarray(df.price, dtype=np.float64)
        players = np.asarray(df.player)
        points = np.asarray(df.points)
        positions = np.asarray(df.pos)
        num_players = players.shape[0]
        num_rows = len(row_dict.keys())
        mip = glp_create_prob();  #create problem instance
        glp_set_prob_name(mip, "ffopt");  # set nmame
        glp_set_obj_dir(mip, GLP_MAX);  #set to maximize

        glp_add_rows(mip, num_rows);  #add three rows



        #add three rows named p, q, and r and make them the dot product of the column soln times the row p to be between 0 and 100, etc.''

        for idx, name in enumerate(row_dict.keys()):
            glp_set_row_name(mip, idx + 1, name);
            glp_set_row_bnds(mip, idx + 1, GLP_UP, 0, row_dict[name]);
        # glp_set_row_name(mip, 2, "q");
        # glp_set_row_bnds(mip, 2, GLP_UP, 0.0, 600.0);
        # glp_set_row_name(mip, 3, "r");
        # glp_set_row_bnds(mip, 3, GLP_UP, 0.0, 300.0);

        #add num_players columns (the variables we solve for)
        glp_add_cols(mip, num_players);

        #for each plater set the variable to binary and set the obj fxn coefficient to the points
        for idx in range(num_players):
            glp_set_col_name(mip, idx + 1, players[idx]);
            glp_set_col_kind(mip, idx + 1, GLP_BV)
            glp_set_obj_coef(mip, idx + 1, points[idx]);

        ia = intArray(num_players * num_rows + 1);
        ja = intArray(num_players * num_rows + 1);
        ar = doubleArray(num_players * num_rows + 1);

        counter = 1
        for row_idx, row in enumerate(row_dict.keys()):


            if row == 'Budget':
                for col in range(num_players):
                    ia[counter] = row_idx + 1;
                    ja[counter] = col + 1;
                    ar[counter] = prices[col];
                    counter += 1

            elif row == 'Tot':
                for col in range(num_players):
                    ia[counter] = row_idx + 1;
                    ja[counter] = col + 1;
                    ar[counter] = 1.0;
                    counter += 1

            else:
                for col in range(num_players):
                    ia[counter] = row_idx + 1;
                    ja[counter] = col + 1;
                    ar[counter] = 0.0 if positions[col] != row else 1.0
                    counter += 1




        #setup matrix
        glp_load_matrix(mip, num_rows * num_players, ia, ja, ar);
        parm = glp_iocp()
        parm.presolve = GLP_ON
        parm.msg_lev = GLP_MSG_OFF
        parm.br_tech = GLP_BR_DTH
        parm.bt_tech = GLP_BT_BLB
        parm.pp_tech = GLP_PP_ALL
        parm.fp_heur = GLP_OFF
        parm.gmi_cuts = GLP_OFF
        parm.mir_cuts = GLP_OFF
        parm.cov_cuts = GLP_OFF
        parm.clq_cuts = GLP_OFF
        parm.tol_parm = 1e-5
        parm.tol_obj = 1e-7
        parm.mip_gap = 0.0
        parm.tol_int = 1e-5
        parm.tm_lim = 65000
        parm.out_frq = 5000
        parm.out_dly = 10000
        parm.cb_size = 0
        parm.binarize = GLP_OFF

        glp_intopt(mip, parm)
        #run simplex
        # glp_simplex(mip, None);

        #get objective value
        Z = glp_mip_obj_val(mip);

        player_mask = np.ones((num_players,)) < 1
        #get column solutions
        for i in range(num_players):
            x = glp_mip_col_val(mip, i + 1);
            if not x < 1:
                player_mask[i] = True

#         if not quiet:
#             print my_points
#             print Z
        aths = []
        for pos in pos_list:
            temp_mask = positions[player_mask] == pos
            plyrs = players[player_mask][temp_mask]
            prcs = prices[player_mask][temp_mask]
            athletes = zip(plyrs, prcs,len(plyrs)*[pos])
            aths.extend(athletes)
            for ath in athletes:
                pl, price, pos = ath
                if not quiet:
                    print pos + ': ', pl, '$' + str(int(price))
        if not quiet:
            print " "

#         if not quiet:
#             print my_points + Z
        glp_delete_prob(mip);
        return aths


def get_best_bench():
    row_dict = {'QB': 2, 'RB': 2, 'WR': 3, 'TE': 1, 'K': 1, 'Budget': 16.0, 'Tot': 5}
    f = ff_opt(row_dict=row_dict)
    return f

#loop thru and say they_got the 10 best qb's, the 20 best rb's

In [54]:
ff = ff_opt()

they got:  antoniobrown
QB:  aaronrodgers $18
RB:  le'veonbell $57
RB:  ameerabdullah $3
RB:  demarcomurray $37
TE:  tylereifert $2
WR:  dezbryant $32
WR:  goldentate $8
WR:  jarvislandry $5
K:  stephengostkowski $1
 
backup  QB   mattryan  $ 6
backup  RB   ameerabdullah  $ 3
backup  RB   leseanmccoy  $ 55
backup  RB   demarcomurray  $ 37
backup  RB   le'veonbell  $ 57
backup  RB   tymontgomery  $ 10
backup  RB   demarcomurray  $ 37
backup  RB   le'veonbell  $ 57
backup  RB   ameerabdullah  $ 3
backup  RB   leseanmccoy  $ 55
backup  TE   gregolsen  $ 10
backup  WR   amaricooper  $ 27
backup  WR   goldentate  $ 8
backup  WR   jarvislandry  $ 5
backup  WR   dezbryant  $ 32
backup  WR   jarvislandry  $ 5
backup  WR   larryfitzgerald  $ 10
backup  WR   amaricooper  $ 27
backup  WR   goldentate  $ 8
backup  WR   larryfitzgerald  $ 10
backup  K   justintucker  $ 1
they got:  tymontgomery
QB:  aaronrodgers $18
RB:  le'veonbell $57
RB:  ameerabdullah $3
RB:  demarcomurray $37
TE:  tylereifert 

backup  RB   demarcomurray  $ 37
backup  RB   le'veonbell  $ 57
backup  RB   ameerabdullah  $ 3
backup  RB   leseanmccoy  $ 55
backup  TE   gregolsen  $ 10
backup  WR   goldentate  $ 8
backup  WR   jarvislandry  $ 5
backup  WR   tyhilton  $ 37
backup  WR   dezbryant  $ 32
backup  WR   jarvislandry  $ 5
backup  WR   larryfitzgerald  $ 10
backup  WR   ericdecker  $ 3
backup  WR   dezbryant  $ 32
backup  WR   goldentate  $ 8
backup  K   justintucker  $ 1
they got:  jordanhoward
QB:  aaronrodgers $18
RB:  le'veonbell $57
RB:  ameerabdullah $3
RB:  demarcomurray $37
TE:  tylereifert $2
WR:  dezbryant $32
WR:  goldentate $8
WR:  jarvislandry $5
K:  stephengostkowski $1
 
backup  QB   mattryan  $ 6
backup  RB   ameerabdullah  $ 3
backup  RB   leseanmccoy  $ 55
backup  RB   demarcomurray  $ 37
backup  RB   le'veonbell  $ 57
backup  RB   christianmccaffrey  $ 11
backup  RB   demarcomurray  $ 37
backup  RB   le'veonbell  $ 57
backup  RB   ameerabdullah  $ 3
backup  RB   leseanmccoy  $ 55
backup 